# Assignment 1: Practical Deep Learning for Language Processing (DS405B)
submitted by Tim-Moritz Bündert (ID: 5635975) on November 23, 2021

In [ ]:
# mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# set data directory
data_dir = "/content/drive/MyDrive/University/DS405B_PDL_for_LP/Assignment_1/vocab/"

# Part A - Learning Word2Vec on Custom Dataset using Continuous Bag of Words

## Installing and loading packages

In [ ]:
!pip install datasets

     |████████████████████████████████| 290 kB 4.9 MB/s 
     |████████████████████████████████| 132 kB 52.7 MB/s 
     |████████████████████████████████| 59 kB 6.4 MB/s 
     |████████████████████████████████| 243 kB 46.4 MB/s 
     |████████████████████████████████| 1.1 MB 33.6 MB/s 
     |████████████████████████████████| 271 kB 54.4 MB/s 
     |████████████████████████████████| 160 kB 53.5 MB/s 
     |████████████████████████████████| 192 kB 58.1 MB/s 


In [ ]:
from datasets import load_dataset
import torchtext
import torch
import torch.nn as nn
import numpy as np
import random
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

## Load data and tokenizer

In [ ]:
data_train = load_dataset('ag_news', split=('train'))

Downloading:   0%|          | 0.00/1.83k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

Using custom data configuration default


Downloading:   0%|          | 0.00/11.0M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/751k [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset ag_news downloaded and prepared to /root/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548. Subsequent calls will reuse this data.


In [ ]:
tokenizer = torchtext.data.get_tokenizer("basic_english")
tokens = tokenizer('This is an example sentence.') # ensure that tokenizer works
tokens

['this', 'is', 'an', 'example', 'sentence', '.']

## Build custom PyTorch `Dataset` class

In [ ]:
class TextDataset(Dataset):
   def __init__(self, text_data):
     self.samples = text_data
     
   def __len__(self):
     return len(self.samples)

   def __getitem__(self,idx):
     return(self.samples[idx])
    
dataset_train = TextDataset(data_train["text"]) # build class from the imported Hugging Face dataset

## Create vocabulary

In [ ]:
vocab = torchtext.vocab.build_vocab_from_iterator(map(tokenizer, dataset_train), specials=["<unk>"], min_freq=50)
vocab.set_default_index(vocab["<unk>"])

In [ ]:
vocab(['here', 'is', 'an', 'example'])

[475, 21, 30, 5297]

## Set up Dataloader

In [ ]:
WINDOW_SIZE = 11
MAX_SEQUENCE_LENGTH = 256

def collate_batches_BOW(batch):
  batch_input, batch_output = [], []
  for text in batch:
    text_token_ids = vocab(tokenizer(text)) # transform text into sequence of token IDs

    if len(text_token_ids) < WINDOW_SIZE:
      continue

    text_token_ids = text_token_ids[:MAX_SEQUENCE_LENGTH]

    for idx in range(len(text_token_ids) - (WINDOW_SIZE - 1)):
      token_id_sequence = text_token_ids[idx: idx+WINDOW_SIZE]
      output = token_id_sequence.pop(WINDOW_SIZE//2)
      input = token_id_sequence

      batch_input.append(input)
      batch_output.append(output)

  batch_input = torch.tensor(batch_input, dtype=torch.long)
  batch_output = torch.tensor(batch_output, dtype=torch.long)
  return batch_input, batch_output

train_dataloader = DataLoader(dataset_train, batch_size=96, shuffle=True, collate_fn=collate_batches_BOW)

## Define `word2vecModel`

In [ ]:
class word2vecModel(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()

    self.embeddings = nn.Embedding(num_embeddings=vocab_size, embedding_dim=300) 
    self.linear = nn.Linear(in_features=300, out_features=vocab_size)

  def forward(self, x):
    x = self.embeddings(x)
    x = x.mean(axis=1) # for CBOW: averaging of context (input) words, see https://arxiv.org/pdf/1301.3781.pdf [alternative: use EmbeddingBag]
    x = self.linear(x)
    return x

## Training of the CBOW model

In [ ]:
model = word2vecModel(vocab_size=len(vocab.get_stoi()))
device = torch.device("cuda")
model.to(device)

word2vecModel(
  (embeddings): Embedding(7934, 300)
  (linear): Linear(in_features=300, out_features=7934, bias=True)
)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

num_epochs = 15
for epoch in range(num_epochs):
  running_loss = []
  for batch_data in train_dataloader:
    inputs = batch_data[0].to(device)
    labels = batch_data[1].to(device)

    optimizer.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    running_loss.append(loss.item())

  print(f'Epoch: {epoch} Training Loss: {np.mean(running_loss)}')

  # Note: experimenting with different window sizes (i.e., 7, 9, 11) and learning rates (i.e., 0.01, 0.02), all models performed similarly in terms of the training loss. 
  # The present configuration yielded the (slightly) best performance in terms of the training loss.

Epoch: 0 Training Loss: 4.939947088241577
Epoch: 1 Training Loss: 4.266551254463196
Epoch: 2 Training Loss: 4.05287971534729
Epoch: 3 Training Loss: 3.9275704530715942
Epoch: 4 Training Loss: 3.84435050239563
Epoch: 5 Training Loss: 3.784074138069153
Epoch: 6 Training Loss: 3.738144139289856
Epoch: 7 Training Loss: 3.702759964752197
Epoch: 8 Training Loss: 3.674602738380432
Epoch: 9 Training Loss: 3.65082503528595
Epoch: 10 Training Loss: 3.6314190202713013
Epoch: 11 Training Loss: 3.615622275733948
Epoch: 12 Training Loss: 3.601422473716736
Epoch: 13 Training Loss: 3.5891007692337036
Epoch: 14 Training Loss: 3.578954795837402


## Retrieve embeddings and list nearest neighbors of seed words

In [ ]:
embeddings = list(model.parameters())[0]
embeddings = embeddings.cpu().detach().numpy()

In [ ]:
def cosine_similarity(word_vec, words_mat): # equivalent to torch.nn.CosineSimilarity(dim=1, eps=1e-6)
  word_vec = np.reshape(word_vec, (len(word_vec), 1))
  nominator = np.matmul(words_mat, word_vec).flatten() # compute inner product of word_vec with each row (word vector) of words_mat

  word_vec_norm = np.sqrt(np.sum(word_vec**2))
  words_mat_norms = np.sqrt(np.sum(words_mat**2, axis = 1))

  denominator = word_vec_norm * words_mat_norms

  cosine_similarities = np.true_divide(nominator, denominator)
  return cosine_similarities

In [ ]:
def nearest_neighbours(word: str, topN: int = 10):
    word_id = vocab[word]
    if word_id == 0:
        print("Out of vocabulary word")
        return

    word_vec = embeddings[word_id]
    cos_sim = cosine_similarity(word_vec, embeddings) # compute cosine similarity of word_vec with all words in vocabulary

    topN_ids = np.argsort(-cos_sim)[1 : topN + 1]

    topN_dict = {}
    for sim_word_id in topN_ids:
        sim_word = vocab.lookup_token(sim_word_id)
        topN_dict[sim_word] = cos_sim[sim_word_id]
    return topN_dict

In [ ]:
random.seed(42)

seed_words = [vocab.lookup_token(random.randint(0, len(vocab))) for p in range(0, 5)]
print(f'Randomly selected words: {seed_words}')

Randomly selected words: ['trademark', 'rebel', '10', 'texans', 'islamabad']


In [ ]:
for seed_word in seed_words:
  print(f'Seed word: {seed_word} \nNearest neighbours: {[word for word, sim in nearest_neighbours(seed_word).items()]} \n')

Seed word: trademark 
Nearest neighbours: ['peers', 'pause', 'confidence', 'flagship', 'netflix', 'portfolio', 'bid-rigging', 'hollinger', 'dollar', 'amazon'] 

Seed word: rebel 
Nearest neighbours: ['rebels', 'insurgent', 'guerrilla', 'judgment', 'insurgents', 'rebel-held', 'militant', 'militia', 'insurgency', 'separatist'] 

Seed word: 10 
Nearest neighbours: ['20', '15', '14', '40', '30', 'eight', '50', 'five', 'seven', 'nine'] 

Seed word: texans 
Nearest neighbours: ['lions', 'astros', 'jaguars', 'pistons', 'bucks', 'bowl', 'yankees', 'lakers', 'tigers', 'offseason'] 

Seed word: islamabad 
Nearest neighbours: ['kabul', 'washington', 'athens', 'tehran', 'moscow', 'rome', 'london', 'kathmandu', 'port-au-prince', 'dhaka'] 



# Part B - Movie Genre Classification using Pre-trained Word Vectors

## Loading packages

In [ ]:
import torchtext
import nltk
from nltk.corpus import stopwords 
import numpy as np
import pandas as pd
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

## Defining genres and movies and removing stopwords from movie titles

In [ ]:
genres = ['action', 'adventure', 'comedy', 'drama', 'fantasy', 'horror', 'romance', 'thriller']
movies = ['The Hangover', 'Shutter Island', 'Fight Club', 'Jumanji', 'Narcos', 'The Matrix', 'Rush Hour', 'The Mummy', 'Iron Man', 'Silence of the Lambs', 'Batman Begins', 'Spider Man', 'The Hobbit', 'Troy', 'Jurassic Park', 'Scary Movie', 'Mission Impossible', 'Ted', 'Eat Pray Love', 'The Notebook', 'Love Actually', 'The Terminal', 'Crazy Stupid Love', 'Twilight', 'The Martian', 'Pursuit of Happyness']

In [ ]:
tokenizer = torchtext.data.get_tokenizer("basic_english")

movies_filtered = []
for movie in movies: 
  filtered_words = [word for word in tokenizer(movie) if word.lower() not in set(stopwords.words("english"))] # remove stopwords
  movies_filtered.append(filtered_words)

print(movies_filtered)

[['hangover'], ['shutter', 'island'], ['fight', 'club'], ['jumanji'], ['narcos'], ['matrix'], ['rush', 'hour'], ['mummy'], ['iron', 'man'], ['silence', 'lambs'], ['batman', 'begins'], ['spider', 'man'], ['hobbit'], ['troy'], ['jurassic', 'park'], ['scary', 'movie'], ['mission', 'impossible'], ['ted'], ['eat', 'pray', 'love'], ['notebook'], ['love', 'actually'], ['terminal'], ['crazy', 'stupid', 'love'], ['twilight'], ['martian'], ['pursuit', 'happyness']]


## Construct the class `PreTrainedEmbeddings` to evaluate pre-trained embeddings

In [ ]:
class PreTrainedEmbeddings(object):
  def __init__(self, vocab):
    self.vocab = vocab

  def get_word_vec(self, word):
    return self.vocab.vectors[self.vocab.stoi[word]] # retrieve work vector for given word

  def cosine_similarity(self, movie_title, genre): # equivalent to torch.nn.CosineSimilarity(dim=1, eps=1e-6)
    cos_sim = []    
    genre_vec = self.get_word_vec(genre).numpy()

    for word in movie_title:
      word_vec = self.get_word_vec(word).numpy()

      nominator = np.inner(word_vec, genre_vec)

      word_vec_norm = np.sqrt(np.sum(word_vec**2))
      genre_vec_norm = np.sqrt(np.sum(genre_vec**2))

      denominator = word_vec_norm * genre_vec_norm
      
      cos_sim.append(np.true_divide(nominator, denominator))

    return np.average(cos_sim) # output the average cosine similarity of the words in the movie_title with genre
    
  def evaluate_embeddings(self, print_cos_sim = False):
    emb_results = []
    for i in range(len(movies)):
      cos_sim_movie = [self.cosine_similarity(movies_filtered[i], genre) for genre in genres]
      idx_highest_sim = np.argmax(cos_sim_movie) # select index of genre with highest average cosine similarity
      if print_cos_sim:
        print(f'Movie: {movies[i]} \nHighest similarity: {genres[idx_highest_sim]} (cosine similarity = {cos_sim_movie[idx_highest_sim]})\n')
      emb_results.append({'movie': movies[i], 'genre': genres[idx_highest_sim], 'cos_sim': cos_sim_movie[idx_highest_sim]})

    return emb_results

## Loading embeddings

### GloVe: 42B

In [ ]:
glove_42B = torchtext.vocab.GloVe(name='42B', dim=300, cache=data_dir) # options: 300d
print('Loaded {} words'.format(len(glove_42B.itos)))

Loaded 1917494 words


In [ ]:
glove_42B_model = PreTrainedEmbeddings(glove_42B)
glove_42B_results = glove_42B_model.evaluate_embeddings()
del glove_42B, glove_42B_model # free up memory

### GloVe: 840B

In [ ]:
glove_840B = torchtext.vocab.GloVe(name='840B', dim=300, cache=data_dir) # options: 300d
print('Loaded {} words'.format(len(glove_840B.itos)))

Loaded 2196017 words


In [ ]:
glove_840B_model = PreTrainedEmbeddings(glove_840B)
glove_840B_results = glove_840B_model.evaluate_embeddings()
del glove_840B, glove_840B_model # free up memory

### GloVe: twitter.27B - 50d

In [ ]:
glove_twitter_27B_50d = torchtext.vocab.GloVe(name='twitter.27B', dim=50, cache=data_dir) # options: 25d, 50d, 100d, & 200d
print('Loaded {} words'.format(len(glove_twitter_27B_50d.itos)))

Loaded 1193514 words


In [ ]:
glove_twitter_27B_50d_model = PreTrainedEmbeddings(glove_twitter_27B_50d)
glove_twitter_27B_50d_results = glove_twitter_27B_50d_model.evaluate_embeddings()
del glove_twitter_27B_50d, glove_twitter_27B_50d_model # free up memory

### GloVe: twitter.27B - 200d

In [ ]:
glove_twitter_27B_200d = torchtext.vocab.GloVe(name='twitter.27B', dim=200, cache=data_dir) # options: 25d, 50d, 100d, & 200d
print('Loaded {} words'.format(len(glove_twitter_27B_200d.itos)))

Loaded 1193514 words


In [ ]:
glove_twitter_27B_200d_model = PreTrainedEmbeddings(glove_twitter_27B_200d)
glove_twitter_27B_200d_results = glove_twitter_27B_200d_model.evaluate_embeddings()
del glove_twitter_27B_200d, glove_twitter_27B_200d_model # free up memory

### GloVe: 6B - 50d

In [ ]:
glove_6B_50d = torchtext.vocab.GloVe(name='6B', dim=50, cache=data_dir) # options: 50d, 100d, 200d, & 300d
print('Loaded {} words'.format(len(glove_6B_50d.itos)))

Loaded 400000 words


In [ ]:
glove_6B_50d_model = PreTrainedEmbeddings(glove_6B_50d)
glove_6B_50d_results = glove_6B_50d_model.evaluate_embeddings()
del glove_6B_50d, glove_6B_50d_model # free up memory

### GloVe: 6B - 300d

In [ ]:
glove_6B_300d = torchtext.vocab.GloVe(name='6B', dim=300, cache=data_dir) # options: 50d, 100d, 200d, & 300d
print('Loaded {} words'.format(len(glove_6B_300d.itos)))

Loaded 400000 words


In [ ]:
glove_6B_300d_model = PreTrainedEmbeddings(glove_6B_300d)
glove_6B_300d_results = glove_6B_300d_model.evaluate_embeddings()
del glove_6B_300d, glove_6B_300d_model # free up memory

### FastText

In [ ]:
fasttext = torchtext.vocab.FastText(language='en', cache=data_dir)
print('Loaded {} words'.format(len(fasttext.itos)))

Loaded 2519370 words


In [ ]:
fasttext_model = PreTrainedEmbeddings(fasttext)
fasttext_results = fasttext_model.evaluate_embeddings()
del fasttext, fasttext_model # free up memory

## Evaluation of the different pre-trained embeddings

In [ ]:
results = [glove_42B_results, glove_840B_results, glove_twitter_27B_50d_results, glove_twitter_27B_200d_results, glove_6B_50d_results, glove_6B_300d_results, fasttext_results]

df = pd.DataFrame(data={'Movie':[results[0][idx]['movie'] for idx in range(len(results[0]))],
                        
                        # GloVe 42B
                        'Genre1':[glove_42B_results[idx]['genre'] for idx in range(len(glove_42B_results))],
                        'Cosine sim.1':[glove_42B_results[idx]['cos_sim'] for idx in range(len(glove_42B_results))],

                        # GloVe 840B
                        'Genre2':[glove_840B_results[idx]['genre'] for idx in range(len(glove_840B_results))],
                        'Cosine sim.2':[glove_840B_results[idx]['cos_sim'] for idx in range(len(glove_840B_results))],

                         # GloVe twitter.27B - 50d
                        'Genre3':[glove_twitter_27B_50d_results[idx]['genre'] for idx in range(len(glove_twitter_27B_50d_results))],
                        'Cosine sim.3':[glove_twitter_27B_50d_results[idx]['cos_sim'] for idx in range(len(glove_twitter_27B_50d_results))],

                        # GloVe twitter.27B - 200d
                        'Genre4':[glove_twitter_27B_200d_results[idx]['genre'] for idx in range(len(glove_twitter_27B_200d_results))],
                        'Cosine sim.4':[glove_twitter_27B_200d_results[idx]['cos_sim'] for idx in range(len(glove_twitter_27B_200d_results))],

                        # GloVe 6B - 50d
                        'Genre5':[glove_6B_50d_results[idx]['genre'] for idx in range(len(glove_6B_50d_results))],
                        'Cosine sim.5':[glove_6B_50d_results[idx]['cos_sim'] for idx in range(len(glove_6B_50d_results))],

                        # GloVe 6B - 300d
                        'Genre6':[glove_6B_300d_results[idx]['genre'] for idx in range(len(glove_6B_300d_results))],
                        'Cosine sim.6':[glove_6B_300d_results[idx]['cos_sim'] for idx in range(len(glove_6B_300d_results))],

                        # FastText
                        'Genre7':[fasttext_results[idx]['genre'] for idx in range(len(fasttext_results))],
                        'Cosine sim.7':[fasttext_results[idx]['cos_sim'] for idx in range(len(fasttext_results))]
                        
                        }).set_index('Movie')
df
df.columns = [list(np.concatenate([2*['GloVe 42B (300d)'], 2*['GloVe 840B (300d)'], 2*['GloVe twitter.27B (50d)'], 2*['GloVe twitter.27B (200d)'], 2*['GloVe 6B (50d)'], 2*['GloVe 6B (300d)'], 2*['FastText (300d)']])), 
              list(np.concatenate([len(results)*['Genre', 'Cosine sim.']]))] # set column and subcolumn names
df.index.name = ''

df

GloVe 42B (300d)              ... FastText (300d)            
                                Genre Cosine sim.  ...           Genre Cosine sim.
                                                   ...                            
The Hangover                   comedy    0.368060  ...          comedy    0.366995
Shutter Island              adventure    0.300728  ...       adventure    0.232774
Fight Club                     action    0.481213  ...       adventure    0.228288
Jumanji                        horror    0.161638  ...       adventure    0.498415
Narcos                         horror   -0.035211  ...        thriller    0.338854
The Matrix                     action    0.362815  ...          action    0.200232
Rush Hour                      action    0.383655  ...           drama    0.261956
The Mummy                      horror    0.364619  ...          horror    0.443330
Iron Man                       action    0.396015  ...       adventure    0.213270
Silence of the Lambs           horror    0.281280  ...        thriller    0.321620
Batman Begins                  action    0.450267  ...       adventure    0.297119
Spider Man                     action    0.345434  ...       adventure    0.271150
The Hobbit                     horror    0.356336  ...         fantasy    0.443128
Troy                          fantasy    0.236850  ...         romance    0.214627
Jurassic Park               adventure    0.391265  ...       adventure    0.309011
Scary Movie                    horror    0.591638  ...          horror    0.577309
Mission Impossible             action    0.438949  ...       adventure    0.240337
Ted                            comedy    0.354868  ...          comedy    0.199814
Eat Pray Love                  action    0.342377  ...         romance    0.296370
The Notebook                  fantasy    0.242729  ...         fantasy    0.253339
Love Actually                  action    0.466041  ...         romance    0.380516
The Terminal                   action    0.240037  ...         fantasy    0.161870
Crazy Stupid Love              comedy    0.419910  ...          comedy    0.384747
Twilight                      romance    0.508341  ...         fantasy    0.484695
The Martian                 adventure    0.261474  ...         fantasy    0.312986
Pursuit of Happyness        adventure    0.291463  ...       adventure    0.365553

[26 rows x 14 columns]

Based on the above comparison, all of the pre-trained word embeddings provide reasonable results with suitable classifications for most of the movies.

Subjectively, I would tend to think that the `FastText` embeddings worked overall best considering the suitable classication of more difficult movies such as *The Hangover* and *Spider Man*. In particular, `FastText` performs well for the genre *comedy* with movies such as *Ted* or *Crazy Stupid Love*.
Also,  `GloVe 42B (300d)` yields a very good performance.

On the other hand, `GloVe 840B (300d)`, for example, performs well on genres such as *adventure* and *action*.

Furthermore, when considering the pre-trained embeddings which were evaluated both using a smaller and a higher dimensionality of word vectors, both word dimensionalities predominantly lead to the same predicted genre. However, it can be noticed that the embeddings with lower dimensionality usually provide a higher cosine similarity for that genre as does its higher-dimensional counterpart.

In general, it is important to consider that this task is inherently subjective as also humans would disagree regarding the specific genre of some movies. In addition, the title alone may not suffice to infer the genre of a film.